# Monte Carlo

### Model objects

BioSTEAM streamlines uncertainty analysis with an object-oriented framework where a [Model](../evaluation/Model.txt) object samples from parameter distributions and reevaluates biorefinery metrics at each new condition. In essence, a Model object sets parameter values, simulates the biorefinery system, and evaluates metrics across an array of samples.

![Simple Model](Model_Simple_UML.png "Model Simple UML")

Model objects are able to cut down simulation time by sorting the samples to minimize perturbations to the system between simulations. This decreases the number of iterations required to solve recycle systems. The following examples show how Model objects can be used.


### Create parameter distributions

**Let's first learn how to create common parameter distributions using** [chaospy](https://chaospy.readthedocs.io/en/master/tutorial.html).

A triangular distribution is typically used when the parameter is uncertain within given limits, but is heuristically known to take a particular value. Create a triangular distribution:

In [1]:
from chaospy import distributions as shape
lower_bound = 0
most_probable = 0.5
upper_bound = 1
triang = shape.Triangle(lower_bound, most_probable, upper_bound)
print(triang)

Triangle(lower=0, midpoint=0.5, upper=1)


A uniform distribution is used when the theoretical limits of the parameter is known, but no information is available to discern which values are more probable. Create a uniform distribution:

In [2]:
from chaospy import distributions as shape
lower_bound = 0
upper_bound = 1
unif = shape.Uniform(lower_bound, upper_bound)
print(unif)

Uniform(lower=0, upper=1)


A large set of distributions are available through chaospy, but generally triangular and uniform distributions are the most widely used to describe the uncertainty of parameters in Monte Carlo analyses.

### Parameter objects

**Parameter objects are simply structures BioSTEAM uses to manage parameter values and distributions.**

This section is just to get you familiar with Parameter objects. All the fields that a Parameter object can have are described below. Don't worry if you don't fully understand what each field does. The main idea is that we need to define the `setter` function that the Parameter object uses to set the parameter value to the `element` (e.g. unit operation, stream, etc.) it pertains to. We can also pass a `distribution` (i.e. a chaospy distribution) that will be accessible for Model objects to sample from. As for the `name`, `units` of measure, and the `baseline` value, these are all for bookkeeping purposes. BioSTEAM incorporates the `name` and `units` of measure when creating a DataFrame of Monte Carlo results and parameter distributions. Parameter objects are created by Model objects which implicitly pass both the `system` affected by the parameter, and the `simulate` function. So don't worry about these last two fields, they are automatically added by the Model object when creating the parameter.

**simulate:** [function] Should simulate parameter effects.

**system:** [System] System associated to parameter.

**name:** [str] Name of parameter.

**units:** [str] Units of measure.

**baseline:** [float] Baseline value of parameter.

**element:** [object] Element associated to parameter.

**setter:** [function] Should set the parameter.

**distribution:** [chaospy.Dist] Parameter distribution.

Hopefully things will be become clearer as we start to create the parameter objects in the following sections...
    

### Create a model object

**Model objects are used to evaluate metrics around multiple parameters of a system.**

Create a Model object of the lipidcane biorefinery with internal rate of return and utility cost as metrics:

In [3]:
from biorefineries import lipidcane as lc
import biosteam as bst
solve_IRR = lc.lipidcane_tea.solve_IRR
total_utility_cost = lambda: lc.lipidcane_tea.utility_cost / 10**6 # In 10^6 USD/yr
metrics = (bst.Metric('Internal rate of return', lc.lipidcane_tea.solve_IRR, '%'),
           bst.Metric('Utility cost', total_utility_cost, '10^6 USD/yr'))
model = bst.Model(lc.lipidcane_sys, metrics)


The Model object begins with no parameters: 

In [4]:
model

Model: Biorefinery internal rate of return (%)
       Biorefinery utility cost (10^6 USD/yr)
 (No parameters)


### Add design parameters

**A design parameter is a Unit attribute that changes design requirements but does not affect mass and energy balances.**

Add number of fermentation reactors as a "design" parameter:

In [5]:
R301 = bst.main_flowsheet.unit.R301 # The Fermentation Unit
@model.parameter(name='Number of reactors',
                 element=R301, kind='design',
                 distribution=shape.Uniform(4, 10))
def set_N_reactors(N):
    R301.N = round(N)

The decorator uses the function to create a Parameter object and add it to the model:

In [6]:
parameters = model.get_parameters()
parameters

(<Parameter: [Fermentation-R301] Number of reactors>,)

Calling a Parameter object will update the parameter and results:

In [7]:
set_N_reactors_parameter = parameters[0]
set_N_reactors_parameter(5)
print(f'Puchase cost at 5 reactors: ${R301.purchase_cost:,.0f}')
set_N_reactors_parameter(8)
print(f'Puchase cost at 8 reactors: ${R301.purchase_cost:,.0f}')

Puchase cost at 5 reactors: $1,718,933
Puchase cost at 8 reactors: $2,030,639


The distribution will come into play later, when creating samples for Monte Carlo simulations.

### Add cost parameters

**A cost parameter is a Unit attribute that affects cost but does not change design requirements.**

Add the fermentation unit base cost as a "cost" parameter with a triangular distribution:

In [8]:
reactors_cost_coefficients = R301.cost_items['Reactors']
mid = reactors_cost_coefficients.n # Most probable at baseline value
lb = mid - 0.1 # Minimum
ub = mid + 0.1 # Maximum
@model.parameter(element=R301, kind='cost',
                 distribution=shape.Triangle(lb, mid, ub))
def set_exponential_cost_coefficient(exponential_cost_coefficient):
    reactors_cost_coefficients.n = exponential_cost_coefficient

Note that if the name was not defined, it defaults to the setter's signature:

In [9]:
model.get_parameters()

(<Parameter: [Fermentation-R301] Number of reactors>,
 <Parameter: [Fermentation-R301] Exponential cost coefficient>)

### Add isolated parameters

**An isolated parameter should not affect Unit objects in any way.**

Add feedstock price as a "isolated" parameter:

In [10]:
lipidcane = lc.lipidcane # The feedstock stream
lb = lipidcane.price * 0.9 # Minimum price
ub = lipidcane.price * 1.1 # Maximum price
@model.parameter(element=lipidcane, kind='isolated', units='USD/kg',
                 distribution=shape.Uniform(lb, ub))
def set_feed_price(feedstock_price):
    lipidcane.price = feedstock_price

### Add coupled parameters

**A coupled parameter affects mass and energy balances of the system.**

Add lipid fraction as a "coupled" parameter:

In [11]:
from biorefineries.lipidcane.utils import set_lipid_fraction
# Note that if the setter function is already made,
# you can pass it as the first argument
set_lipid_fraction = model.parameter(set_lipid_fraction,
                                     element=lipidcane, kind='coupled',
                                     distribution=shape.Uniform(0.05, 0.10))

Add fermentation efficiency as a "coupled" parameter:

In [12]:
@model.parameter(element=R301, kind='coupled',
                 distribution=shape.Triangle(0.85, 0.90, 0.95))
def set_fermentation_efficiency(efficiency):
    R301.efficiency = efficiency

### Evaluate metric given a sample

**The model can be called to evaluate a sample of parameters.**

Note that all parameters are stored in the model with highly coupled parameters first:

In [13]:
model

Model: Biorefinery internal rate of return (%)
       Biorefinery utility cost (10^6 USD/yr)
 Element:           Parameter:
 Stream-lipidcane   Lipid fraction
 Fermentation-R301  Efficiency
                    Number of reactors
                    Exponential cost coefficient
 Stream-lipidcane   Feedstock price


Get dictionary that contain DataFrame objects of parameter distributions:

In [14]:
df_dct = model.get_distribution_summary()
df_dct['Uniform']

,Element,Name,Units,Shape,lower,upper
0,Stream-lipidcane,Lipid fraction,,Uniform,0.05,0.1
1,Fermentation-R301,Number of reactors,,Uniform,4,10
2,Stream-lipidcane,Feedstock price,USD/kg,Uniform,0.0311,0.038


In [15]:
df_dct['Triangle']

,Element,Name,Units,Shape,lower,midpoint,upper
0,Fermentation-R301,Efficiency,,Triangle,0.85,0.9,0.95
1,Fermentation-R301,Exponential cost coefficient,,Triangle,0.4,0.5,0.6


Evaluate sample:

In [16]:
model([0.05, 0.85, 8, 100000, 0.040]) # Returns metrics (IRR and utility cost)

{('Biorefinery', 'Internal rate of return [%]'): 0.10947587895790568,
 ('Biorefinery', 'Utility cost [10^6 USD/yr]'): -17.975099677963602}

### Monte Carlo

Sample from a joint distribution, and simulate samples:

In [17]:
N_samples = 100
rule = 'L' # For Latin-Hypercube sampling
samples = model.sample(N_samples, rule)
model.load_samples(samples)
model.evaluate()
model.table # All evaluations are stored as a pandas DataFrame

Element  Stream-lipidcane Fermentation-R301                     \
Variable   Lipid fraction        Efficiency Number of reactors   
0                  0.0507               0.9               4.67   
1                  0.0742             0.916                8.3   
2                  0.0709              0.89               4.32   
3                  0.0878             0.906               4.37   
4                  0.0849             0.943               4.88   
..                    ...               ...                ...   
95                 0.0806             0.862                8.6   
96                 0.0827             0.923               9.11   
97                 0.0636             0.919               6.58   
98                 0.0595             0.887               6.31   
99                 0.0607             0.927               9.31   

Element                                       Stream-lipidcane  \
Variable Exponential cost coefficient Feedstock price [USD/kg]   
0                               0.484                   0.0355   
1                               0.489                   0.0363   
2                               0.497                   0.0331   
3                               0.538                   0.0319   
4                               0.565                   0.0318   
..                                ...                      ...   
95                              0.441                   0.0339   
96                              0.539                   0.0365   
97                               0.49                   0.0367   
98                              0.486                   0.0347   
99                              0.521                   0.0331   

Element                  Biorefinery                             
Variable Internal rate of return [%] Utility cost [10^6 USD/yr]  
0                              0.143                      -17.5  
1                              0.158                      -21.6  
2                              0.169                      -21.2  
3                              0.188                      -24.1  
4                              0.191                      -23.3  
..                               ...                        ...  
95                             0.165                      -23.2  
96                             0.164                        -23  
97                              0.15                      -19.6  
98                             0.152                      -19.2  
99                             0.168                        -19  

[100 rows x 7 columns]

Note that coupled parameters are on the left most columns, and are ordered from upstream to downstream (e.g. <Stream: Lipid cane> is upstream from <Fermentation: R301>)

### Behind the scenes

![Model UML Diagram](Model_UML.png "Model UML")

Model objects work with the help of Block and Parameter objects that are able to tell the relative importance of parameters through the `element` it affects and the `kind` (how it affects the system). Before a new parameter is made, if its `kind` is "coupled", then the Model object creates a Block object that simulates only the objects affected by the parameter. The Block object, in turn, helps to create a Parameter object by passing its simulation method.